In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import sklearn as sk
import os
import random
import math

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)

print("Num Physical Devices Available: ", len(physical_devices))

print(tf.__version__)
print(cv2.__version__)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num Physical Devices Available:  1
2.8.0
4.6.0


**Constants**


In [4]:
IMG_SIZE = 600
BATCH_SIZE = 64
#PATH = "C:/Python/chen22/hitting_behaviors/video_data/violence_dataset/"
PATH = "video_data/violence_dataset/"




**Make Datatset**


In [5]:
#from sklearn.model_selection import train_test_split

In [11]:
def video_to_frames(p):
    assert os.path.exists(p)
    cap = cv2.VideoCapture(p)
    
    assert cap
    rate = math.floor(cap.get(3))

    ImageFrames = []
    assert cap.isOpened()
    
    '''make sure to figure out how to extract a certain amount of frames so you don't use too much memory'''
    while cap.isOpened():
        #ID = cap.get(1)
        n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        success, image = cap.read()
        #assert success

        if success:
            for frame in range(n_frames):
                if frame % 20 == 0:
                
                    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                    ImageFrames.append(image)
                else: continue

        else: break

            
        
            
    cap.release()
    return ImageFrames
    

In [13]:
CATEGORIES = ["NonViolence", "Violence"]
X_set = []
y_set = []
for category in os.listdir(PATH):
    
    
    path = os.path.join(PATH, category)
    print(path)
    assert os.path.exists(path)


    class_num = CATEGORIES.index(category)
    for i, video in tqdm(enumerate(os.listdir(path)[0:350])):

        #print(path + '/' + video)
        #print(os.path.join(path, video))
        frames = video_to_frames(path + '/' + video)
        
        
        for j, frame in enumerate(frames):
            X_set.append(frame)
            y_set.append(class_num)

            
    print("success")

video_data/violence_dataset/NonViolence


27it [01:41,  3.78s/it]


KeyboardInterrupt: 

In [ ]:
#EffNet = tf.keras.applications.EfficientNetV2B0(include_top=False, weights = 'imageNet', )